# Python script to assemble sentence data

This script takes the JSON output of Amazon word-to-speach, builds its words data into sentences, and reports the starting and ending times for each sentence.  It currently ouputs the results to a CSV file.

TODO: The script assumes that there is only one alternative for each word and punctuation.  If there are more than one, it uses the first one and ignores any additional alternatives.  We need to see how often alternatives occur and if they do, how to determine which alternative to use, or whether to present both alternatives.


## modules needed for the script

In [7]:
import json # library for working with JSON 
import csv  # library to reaad/write/parse CSV files

## variables hard-coded for testing

In [8]:
inputFilePath = '20140801CBS.json'
outputFilePath = 'sentenceTable.csv'
print(inputFilePath)

20140801CBS.json


## read episode text from file generated by text-to-speach

In [9]:
inputFileObject = open(inputFilePath, 'rt', newline='')
jsonString = inputFileObject.read()    # reads from the file object as text
jsonStructure = json.loads(jsonString) # turns the JSON text into a Python dictionary

## traverse the JSON structure to get to the level of items

Here's what the structure of the input JSON file looks like:

```
{
    "jobName": "CBS_segment_2018-08-01-5-6-7-8",
    "accountId": "022001073478",
    "results": {
        "transcripts": [
            {
                "transcript": "Working pop up. Hurry in to get a great deal on a honda 
                // LOTS MORE TEXT IN HERE
                responsible for that."
            }
        ],
        "items": [
            {
                "start_time": "0.0",
                "end_time": "0.35",
                "alternatives": [
                    {
                        "confidence": "1.0000",
                        "content": "Working"
                    }
                ],
                "type": "pronunciation"
            },
            {
                "start_time": "0.35",
                "end_time": "0.63",
                "alternatives": [
                    {
                        "confidence": "0.9972",
                        "content": "pop"
                    }
                ],
                "type": "pronunciation"
            },
            {
                "alternatives": [
                    {
                        "confidence": null,
                        "content": "."
                    }
                ],
                "type": "punctuation"
            }
        ]
    },
    "status": "COMPLETED"
}
```

There are many more items than shown here.  To get to the items level, one must extract the value from the "results" key (a JSON object), then the value from the "items" key (a JSON dictionary).  

In [10]:
items = jsonStructure['results']['items']

## create an empty objects for building the array of sentence data

In [11]:
sentences = []   # empty list to contain sentence data
firstWord = True # a flag indicating that the first encountered word is the start of a sentence

## step through the items and assemble them into sentences

Here is an example sentence:

```
James anthony mason, thank you.

start time: 886.159
end time: 887.859
```

and its JSON:

```
// At this point, the firstWord flag is set to True
            {
                "start_time": "886.159",
                "end_time": "886.599",
                "alternatives": [
                    {
                        "confidence": "1.0000",
                        "content": "James"
                    }
                ],
                "type": "pronunciation"
            },
            
// On processing the first word, firstWord is set to False
// The start time of 886.159 is set as the start time for the sentence.
// The end time of 886.599 is set as the end time for the sentence, but will be written over until ending punctuation

            {
                "start_time": "886.669",
                "end_time": "886.979",
                "alternatives": [
                    {
                        "confidence": "0.9933",
                        "content": "anthony"
                    }
                ],
                "type": "pronunciation"
            },
            {
                "start_time": "886.979",
                "end_time": "887.339",
                "alternatives": [
                    {
                        "confidence": "1.0000",
                        "content": "mason"
                    }
                ],
                "type": "pronunciation"
            },
            
// Since this punctuation is a comma, it's added to the sentence and processing continues.

            {
                "alternatives": [
                    {
                        "confidence": null,
                        "content": ","
                    }
                ],
                "type": "punctuation"
            },
            {
                "start_time": "887.339",
                "end_time": "887.649",
                "alternatives": [
                    {
                        "confidence": "0.9989",
                        "content": "thank"
                    }
                ],
                "type": "pronunciation"
            },
            {
                "start_time": "887.649",
                "end_time": "887.859",
                "alternatives": [
                    {
                        "confidence": "0.9962",
                        "content": "you"
                    }
                ],
                "type": "pronunciation"
            },
            
// When this non-comma punctuation is detected, the final punctuation is added to the sentence.
// The startTime recorded at the beginning (886.669) and the endTime from the final word (887.859) are written to the sentence list.
// The firstWord flag is set to True again so the next word will be used to start a new sentence.

            {
                "alternatives": [
                    {
                        "confidence": null,
                        "content": "."
                    }
                ],
                "type": "punctuation"
            },
```


In [12]:
for item in items:
    
    # if the item is a word, add it to the end of the sentence and reset the sentence end time
    if item['type'] == 'pronunciation':    
        endTime = item['end_time']          # the end time will be reset later if this isn't the last word
        if firstWord:
            startTime = item['start_time']  # if this is the first word in a sentence, record the start time
            sentence = item['alternatives'][0]['content']  # begin a new sentence with the first word
            firstWord = False               # the next word encountered will not be the first word in s sentence
        else:
            sentence += ' ' + item['alternatives'][0]['content']
        
    # if the item is punctuation, figure out if it ends the sentence or not
    elif item['type'] == 'punctuation':
        
        # the punctuation is a comma, so continue the sentence
        if item['alternatives'][0]['content'] == ',':
            sentence += ','
            
        # the punctuation is something else, end the sentence
        else:
            sentence += item['alternatives'][0]['content']   # add final punctuation
            sentences.append([startTime, endTime, sentence]) # add the sentence to the sentence list
            firstWord = True # the next word encountered will be considered the first word of a sentence
    
    # I don't believe that there are any other types of content, but if any show up, print them
    else:
        print(item['type'])  

## output the sentences to a CSV file

Note: in the file open command, utf-8 encoding is explicitly specified so that it doesn't default to the system preferred encoding.  That's cp1252 on PCs.  For Macs, I think utf-8 is the system default.

In [13]:
outputObject = open(outputFilePath, 'wt', newline = '', encoding = 'utf-8')
csvOutput = csv.writer(outputObject, delimiter = ',')
csvOutput.writerow(['startTime', 'endTime', 'sentence'])  # output the header row
for sentence in sentences:
    csvOutput.writerow(sentence)  # output the data for each sentence
outputObject.close()